In [2]:
import numpy as np
import subprocess
from itertools import combinations
from helpers.seq import ab_number as abn
import os
import pandas as pd

# make sure to add ANARCI to Path
os.environ['PATH'] = '/root/miniconda3/envs/anarci/bin:' + os.environ['PATH']

def run_anarci(sequence):
    # Run ANARCI as a subprocess

    result = subprocess.run(['ANARCI', '--sequence', sequence, '--scheme', 'aho'], capture_output=True, text=True)

    sequence_results = result.stdout.split('\n')

    species = None
    e_value = None
    score = None
    heavy_chain = np.array([])
    light_chain = np.array([])

    try: #push this into try as we do not want to stop the program if ANARCI fails. If it fails, it will return an empty arrays and thus not be included in the anarci results and data files.
        if len(sequence_results) > 4:

            blank, species, chain_type, e_value, score, seqstart_index, seqend_index, blank_2 = sequence_results[5].split('|')

            h_seq = []
            l_seq = []
            for row in sequence_results[7:]:
                row = [x for x in row.split(' ') if x != '']
                if (len(row) == 3) and (row[0] == 'H'):       
                    h_seq.append(row[2])
                elif (len(row) == 3) and (row[0] == 'L'):
                    l_seq.append(row[2])

            heavy_chain = np.array(h_seq)
            light_chain = np.array(l_seq)

    except:
        pass
    
    return species, e_value, score, heavy_chain, light_chain


def percent_identity(seq1, seq2):
    """ Compute the percent identity of two strings of equal length. """
    if len(seq1) != len(seq2):
        raise ValueError('Sequences must be the same length.')
    i = 0
    for r1, r2 in zip(seq1, seq2):
        i += int(r1 == r2)
    return i * 100 / len(seq1)

def full_seq_identity(df_anarci_H, df_anarci_KL):
    df_anarci_H = df_anarci_H.copy().set_index('Id')
    df_anarci_KL = df_anarci_KL.copy().set_index('Id')
    df_anarci_H['full_seq_H'] = df_anarci_H.loc[:, '1':].apply(lambda x: ''.join(x), axis=1)
    df_anarci_KL['full_seq_KL'] = df_anarci_KL.loc[:, '1':].apply(lambda x: ''.join(x), axis=1)
    df_anarci = df_anarci_H.merge(df_anarci_KL, left_index=True, right_index=True)
    seqs = [x['full_seq_H'] + x['full_seq_KL'] for _, x in df_anarci.iterrows()]

    identity_dist = [percent_identity(s[0], s[1]) for s in combinations(seqs, 2)]
    return sum(identity_dist) / len(identity_dist)

def cdr3_seq_identity(df_anarci_H):
    df_seqs = df_anarci_H.loc[:, '105':'117']
    seqs = [''.join(x) for _, x in df_seqs.iterrows()]
    identity_dist = [percent_identity(s[0], s[1]) for s in combinations(seqs, 2)]
    return sum(identity_dist) / len(identity_dist)

def diversity_metrics(anarci_csv_path_H, anarci_csv_path_KL):
    df_anarci_H = pd.read_csv(anarci_csv_path_H)
    df_anarci_KL = pd.read_csv(anarci_csv_path_KL)
    results = {'avg_seq_identity_full': full_seq_identity(df_anarci_H, df_anarci_KL),
               'avg_seq_identity_cdr3': cdr3_seq_identity(df_anarci_H)}
    return results

In [23]:
sequence = [
                    "MQIPQAPWPVVWAVLQLGWRPGWFLDSPDRPWNPPTFSPALLVVTEGDNATFTCSFSNTSESFVLNWYRMSPSNQTDKLAAFPEDRSQPGQDCRFRVTQLPNGRDFHMSVVRARRNDSGTYLCGAISLAPKAQIKESLRAELRVTERRAEVPTAHPSPSPRPAGQFQTLVVGVVGGLLGSLVLLVWVLAVICSRAARGTIGARRTGQPLKEDPSAVPVFSVDYGELDFQWREKTPEPPVPCVPEQTEYATIVFPSGMGTSSPARRGSADGPRSAQPLRPEDGHCSWPLGGGGGSGGGGSGGGGSDVQLVESGGDLVKPGGGKLLVPCLASENAKPESLYGKFFKGFVMAGTPAELTLRFENANRYITLFCALQANRAPVARLFRGARVEEGVFRLTALKTDREIPIVYWLAGCPFSGGAIPFLRAQDSLA",
                    "MQIPQAPWPVVWAVLQLGWRPGWFLDSPDRPWNPPTFSPALLVVTEGDNATFTCSFSNTSESFVLNWYRMSPSNQTDKLAAFPEDRSQPGQDCRFRVTQLPNGRDFHMSVVRARRNDSGTYLCGAISLAPKAQIKESLRAELRVTERRAEVPTAHPSPSPRPAGQFQTLVVGVVGGLLGSLVLLVWVLAVICSRAARGTIGARRTGQPLKEDPSAVPVFSVDYGELDFQWREKTPEPPVPCVPEQTEYATIVFPSGMGTSSPARRGSADGPRSAQPLRPEDGHCSWPLGGGGGSGGGGSGGGGSDVQLVESGGDLVKPGGGETGLLQTHLKVQFARPSQQSVMEYRTDVVMMPFAEVRLRNEHGEATTWSRQEAGGWVQRWTQRASQRPRLDATGTVEIAFRIYRSGQPCQVKAEKCDCQVKFRVLEGGLMLVLGCTWQPQQNATAADREETKAFLKGATYWPGAQDSGALPEGAKVDFKLAPGVDVGLSWEYLPGSAKLPNLRASYRGXXXXXEKVSASPLVTEVTNGGLVAPVYKVTSLPKGEGCERSERRIYIAYKQSHFEVCVLTNGTATGPVEGSYH",
                    "MQIPQAPWPVVWAVLQLGWRPGWFLDSPDRPWNPPTFSPALLVVTEGDNATFTCSFSNTSESFVLNWYRMSPSNQTDKLAAFPEDRSQPGQDCRFRVTQLPNGRDFHMSVVRARRNDSGTYLCGAISLAPKAQIKESLRAELRVTERRAEVPTAHPSPSPRPAGQFQTLVVGVVGGLLGSLVLLVWVLAVICSRAARGTIGARRTGQPLKEDPSAVPVFSVDYGELDFQWREKTPEPPVPCVPEQTEYATIVFPSGMGTSSPARRGSADGPRSAQPLRPEDGHCSWPLGGGGGSGGGGSGGGGSDVQLVESGGDLVKPGGGETLRSRELVALSCRVSEDIIGTAGHHYLYWHFLHRVQHASLESHERVPWIPESRDCHINLNSDTVGFPERTGSQILCTVKALDVFDADLAYSPDPLNDLPRAHADSNSNKFLGLLFGGGGSGGGGSGGGGSDVQLVESGGDLVKPGGGETLRSRELVALSCRVSEDIISYAYGANLTTPALSVNGQSTMAPEQLITGLEHEKHQVTISSKPESRTALCSIGLEEGMLISYILTHMECWFCCLEDRVQITREIHNRVVRSKECTVAIVDTNEARNLTLREGLTLNTVLSRGHLICSVLGNSMCHDASILKLGETGHHIAQKFTSGVEVLELELAKYRIVACDNSISRPSKREVETTKMQVLFQAGFQVHAGIKLTPGSSKAKAPVAI",
                    "MQIPQAPWPVVWAVLQLGWRPGWFLDSPDRPWNPPTFSPALLVVTEGDNATFTCSFSNTSESFVLNWYRMSPSNQTDKLAAFPEDRSQPGQDCRFRVTQLPNGRDFHMSVVRARRNDSGTYLCGAISLAPKAQIKESLRAELRVTERRAEVPTAHPSPSPRPAGQFQTLVVGVVGGLLGSLVLLVWVLAVICSRAARGTIGARRTGQPLKEDPSAVPVFSVDYGELDFQWREKTPEPPVPCVPEQTEYATIVFPSGMGTSSPARRGSADGPRSAQPLRPEDGHCSWPLGGGGGSGGGGSGGGGSDVQLVESGGDLVKPGGGPAVQLTVTLHASRSVSLAVLIYRSLTTPLAQPMYSTTVMRECGTSAYTVPEGKSGSPAGPSAEGVYWACTLLFRLRPGSVALQQRLSTSLSLYRTVPSFCVGFHLPPPIISCVSDKEIFSAATTFTNEVTATHLLGRREVDLGTLETDXXAPREVKEPTLEAFKGIRTVSMFRNEVAPFDLCILVLCPQNSAGGGGSGGGGSGGGGSDVQLVQSGAEVKKPGESLKISCKGSGYSFTTYWIGWVRQMPGKGLEWMGIIYPGDSYTKYSPSFQGQVTISADKSISTAYLQWSSLKASDTAMYYCARLSLRVWSGYYYYYYGMDVWGQGTTVTVSSASTKGPSVFPLAPSSKSTSGGTAALGCLVKDYFPEPVTVSWNSGALTSGVHTFPAVLQSSGLYSLSSVVTVPSSSLGTQTYICNVNHKPSNTKVDKRVEPKSCDKTHHHHHHGGGGSGGGGSGGGGSDIQMTQSPSSLSAAVGDKVTITCRASQSFPRYYNYINWYQQQPGKAPKLLIYRASNLWSGVPSRFSGSRSGTEFTLTISSLQPEDFATYYCQQSHEDPYTFGQGTKVEIKRTVAAPSVFIFPPSDEQLKSGTASVVCLLNNFYPREAKVQWKVDNALQSGNSQESVTEQDSKDSTYSLSSTLTLSKADYEKHKVYACEVTHQGLSSPVTKSFNRGEC",
                    "MQIPQAPWPVVWAVLQLGWRPGWFLDSPDRPWNPPTFSPALLVVTEGDNATFTCSFSNTSESFVLNWYRMSPSNQTDKLAAFPEDRSQPGQDCRFRVTQLPNGRDFHMSVVRARRNDSGTYLCGAISLAPKAQIKESLRAELRVTERRAEVPTAHPSPSPRPAGQFQTLVVGVVGGLLGSLVLLVWVLAVICSRAARGTIGARRTGQPLKEDPSAVPVFSVDYGELDFQWREKTPEPPVPCVPEQTEYATIVFPSGMGTSSPARRGSADGPRSAQPLRPEDGHCSWPLGGGGGSGGGGSGGGGSEVQLVESGGDVVKPGGSLRLSCAASGFTFSSYGMSWVRQAPGTGLEWVSVIYGGGRTDYRDDVKGRFTISRDNSKNTLYLQMSSLRAEDTAVYYCAREPPGGDMDYWGQGTLVTVSSASTKGPSVFPLAPSSKSTSGGTAALGCLVKDYFPEPVTVSWNSGALTSGVHTFPAVLQSSGLYSLSSVVTVPSSSLGTQTYICNVNHKPSNTKVDKKVEPKSCGGGGSGGGGSGGGGSYELTQPPSVSVSPGQTARITCSGDALPKQYAYWYQQKPGQAPVLVIYKDSERPSGIPERFSGSSSGTTVTLTISGVQAEDEADYYCQSADSDDINYVFGTGTKVTVLGQPKAAPSVTLFPPSSEELQANKATLVCLISDFYPGAVTVAWKADSSPVKAGVETTTPSKQSNNKYAASSYLSLTPEQWKSHRSYSCQVTHEGSTVEKTVAPTECS"
                ]

# print(species)

df_result_H, df_result_KL = abn.number_seqs_as_df(sequence)


Limiting hmmer search to species ['human', 'mouse'] was requested but hits did not achieve a high enough bitscore. Reverting to using any species
Limiting hmmer search to species ['human', 'mouse'] was requested but hits did not achieve a high enough bitscore. Reverting to using any species
Limiting hmmer search to species ['human', 'mouse'] was requested but hits did not achieve a high enough bitscore. Reverting to using any species


In [40]:
df_result_H.to_dict('records')[0]

{'Id': 'Sequence',
 'domain_no': 0,
 'hmm_species': 'human',
 'chain_type': 'H',
 'e-value': 5.4e-51,
 'score': 163.2,
 'seqstart_index': 528,
 'seqend_index': 654,
 'identity_species': nan,
 'v_gene': nan,
 'v_identity': 0.0,
 'j_gene': nan,
 'j_identity': 0.0,
 '1': 'D',
 '2': 'V',
 '3': 'Q',
 '4': 'L',
 '5': 'V',
 '6': 'Q',
 '7': 'S',
 '8': 'G',
 '9': 'A',
 '10': '-',
 '11': 'E',
 '12': 'V',
 '13': 'K',
 '14': 'K',
 '15': 'P',
 '16': 'G',
 '17': 'E',
 '18': 'S',
 '19': 'L',
 '20': 'K',
 '21': 'I',
 '22': 'S',
 '23': 'C',
 '24': 'K',
 '25': 'G',
 '26': 'S',
 '27': 'G',
 '28': 'Y',
 '29': 'S',
 '30': 'F',
 '31': '-',
 '32': '-',
 '33': '-',
 '34': '-',
 '35': 'T',
 '36': 'T',
 '37': 'Y',
 '38': 'W',
 '39': 'I',
 '40': 'G',
 '41': 'W',
 '42': 'V',
 '43': 'R',
 '44': 'Q',
 '45': 'M',
 '46': 'P',
 '47': 'G',
 '48': 'K',
 '49': 'G',
 '50': 'L',
 '51': 'E',
 '52': 'W',
 '53': 'M',
 '54': 'G',
 '55': 'I',
 '56': 'I',
 '57': 'Y',
 '58': 'P',
 '59': 'G',
 '60': '-',
 '61': '-',
 '62': 'D',
 '

In [27]:
building = b[0]#['hmm_species']

heavy_chain = []
for key in building.keys():
    try:
        int_key = int(key)
        heavy_chain.append(building[key])
    except ValueError:
        # This means the key is not a string representation of a number
        continue

In [35]:
building

{'Id': 'Sequence',
 'domain_no': 0,
 'hmm_species': 'human',
 'chain_type': 'H',
 'e-value': 5.4e-51,
 'score': 163.2,
 'seqstart_index': 528,
 'seqend_index': 654,
 'identity_species': nan,
 'v_gene': nan,
 'v_identity': 0.0,
 'j_gene': nan,
 'j_identity': 0.0,
 '1': 'D',
 '2': 'V',
 '3': 'Q',
 '4': 'L',
 '5': 'V',
 '6': 'Q',
 '7': 'S',
 '8': 'G',
 '9': 'A',
 '10': '-',
 '11': 'E',
 '12': 'V',
 '13': 'K',
 '14': 'K',
 '15': 'P',
 '16': 'G',
 '17': 'E',
 '18': 'S',
 '19': 'L',
 '20': 'K',
 '21': 'I',
 '22': 'S',
 '23': 'C',
 '24': 'K',
 '25': 'G',
 '26': 'S',
 '27': 'G',
 '28': 'Y',
 '29': 'S',
 '30': 'F',
 '31': '-',
 '32': '-',
 '33': '-',
 '34': '-',
 '35': 'T',
 '36': 'T',
 '37': 'Y',
 '38': 'W',
 '39': 'I',
 '40': 'G',
 '41': 'W',
 '42': 'V',
 '43': 'R',
 '44': 'Q',
 '45': 'M',
 '46': 'P',
 '47': 'G',
 '48': 'K',
 '49': 'G',
 '50': 'L',
 '51': 'E',
 '52': 'W',
 '53': 'M',
 '54': 'G',
 '55': 'I',
 '56': 'I',
 '57': 'Y',
 '58': 'P',
 '59': 'G',
 '60': '-',
 '61': '-',
 '62': 'D',
 '

In [37]:
heavy_chain = []
for key in building.keys():
    try:
        int_key = int(key)
        heavy_chain.append(building[key])
    except ValueError:
        # This means the key is not a string representation of a number
        continue

In [42]:
''.join(heavy_chain)

'DVQLVQSGA-EVKKPGESLKISCKGSGYSF----TTYWIGWVRQMPGKGLEWMGIIYPG--DSYTKYSPSFQ-GQVTISADKSISTAYLQWSSLKASDTAMYYCARLSLRVYYGMDVWGQGTTVTVSS'

In [11]:
result = subprocess.run(['ANARCI', '--sequence', sequence, '--scheme', 'aho'], capture_output=True, text=True)

    # sequence_results = result.stdout.split('\n')

print(result) 

CompletedProcess(args=['ANARCI', '--sequence', 'MQIPQAPWPVVWAVLQLGWRPGWFLDSPDRPWNPPTFSPALLVVTEGDNATFTCSFSNTSESFVLNWYRMSPSNQTDKLAAFPEDRSQPGQDCRFRVTQLPNGRDFHMSVVRARRNDSGTYLCGAISLAPKAQIKESLRAELRVTERRAEVPTAHPSPSPRPAGQFQTLVVGVVGGLLGSLVLLVWVLAVICSRAARGTIGARRTGQPLKEDPSAVPVFSVDYGELDFQWREKTPEPPVPCVPEQTEYATIVFPSGMGTSSPARRGSADGPRSAQPLRPEDGHCSWPLGGGGGSGGGGSGGGGSEVQLVESGGDVVKPGGSLRLSCAASGFTFSSYGMSWVRQAPGTGLEWVSVIYGGGRTDYRDDVKGRFTISRDNSKNTLYLQMSSLRAEDTAVYYCAREPPGGDMDYWGQGTLVTVSSASTKGPSVFPLAPSSKSTSGGTAALGCLVKDYFPEPVTVSWNSGALTSGVHTFPAVLQSSGLYSLSSVVTVPSSSLGTQTYICNVNHKPSNTKVDKKVEPKSCGGGGSGGGGSGGGGSYELTQPPSVSVSPGQTARITCSGDALPKQYAYWYQQKPGQAPVLVIYKDSERPSGIPERFSGSSSGTTVTLTISGVQAEDEADYYCQSADSDDINYVFGTGTKVTVLGQPKAAPSVTLFPPSSEELQANKATLVCLISDFYPGAVTVAWKADSSPVKAGVETTTPSKQSNNKYAASSYLSLTPEQWKSHRSYSCQVTHEGSTVEKTVAPTECS', '--scheme', 'aho'], returncode=0, stdout='# Input sequence\n# ANARCI numbered\n# Domain 1 of 2\n# Most significant HMM hit\n#|species|chain_type|e-value|score|seqstart_index|seqend_index|\n#|human|

In [12]:
def full_seq_identity(df_anarci_H, df_anarci_KL):
    df_anarci_H = df_anarci_H.copy().set_index('Id')
    df_anarci_KL = df_anarci_KL.copy().set_index('Id')
    df_anarci_H['full_seq_H'] = df_anarci_H.loc[:, '1':].apply(lambda x: ''.join(x), axis=1)
    df_anarci_KL['full_seq_KL'] = df_anarci_KL.loc[:, '1':].apply(lambda x: ''.join(x), axis=1)
    df_anarci = df_anarci_H.merge(df_anarci_KL, left_index=True, right_index=True)
    seqs = [x['full_seq_H'] + x['full_seq_KL'] for _, x in df_anarci.iterrows()]

    identity_dist = [percent_identity(s[0], s[1]) for s in combinations(seqs, 2)]
    return sum(identity_dist) / len(identity_dist)

In [43]:
full_seq_identity(df_result_H, df_result_KL)

69.72010178117047